In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

# Initialize MediaPipe Pose (for body landmarks)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to overlay clothing
def overlay_clothing(frame, body_landmarks, clothing):
    # Example body landmarks for the t-shirt
    # Replace with actual body landmarks
    left_shoulder = body_landmarks.landmark[5]  # Left shoulder landmark
    right_shoulder = body_landmarks.landmark[2]  # Right shoulder landmark

    # Convert normalized coordinates to pixel values
    h, w, _ = frame.shape
    left_shoulder = (int(left_shoulder.x * w), int(left_shoulder.y * h))
    right_shoulder = (int(right_shoulder.x * w), int(right_shoulder.y * h))

    # Calculate the width of the clothing based on shoulder distance
    clothing_width = int(((right_shoulder[0] - left_shoulder[0]) ** 2 + (right_shoulder[1] - left_shoulder[1]) ** 2) ** 0.5)

    # Resize the clothing to fit the body
    clothing_resized = cv2.resize(clothing, (clothing_width, int(clothing.shape[0] * (clothing_width / clothing.shape[1]))))

    # Calculate the position to place the clothing
    top_left = (left_shoulder[0] - clothing_resized.shape[1] // 2, left_shoulder[1] - clothing_resized.shape[0] // 2)

    # Ensure the overlay does not exceed the frame boundaries
    for i in range(clothing_resized.shape[0]):
        for j in range(clothing_resized.shape[1]):
            y_pos = top_left[1] + i
            x_pos = top_left[0] + j

            if 0 <= x_pos < w and 0 <= y_pos < h and clothing_resized[i, j, 3] > 0:
                frame[y_pos, x_pos] = clothing_resized[i, j, :3]

    return frame

# Load the clothing image (make sure it has an alpha channel)
clothing = cv2.imread(r'E:\jupyter_notebook\Pictures\tshirt1.png', cv2.IMREAD_UNCHANGED)

# Start video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert the frame to RGB for Mediapipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get face landmarks
    face_result = face_mesh.process(rgb_frame)

    # Get body landmarks
    pose_result = pose.process(rgb_frame)

    # If body landmarks are detected, overlay the clothing
    if pose_result.pose_landmarks:
        frame = overlay_clothing(frame, pose_result.pose_landmarks, clothing)

    # Display the result
    cv2.imshow('Virtual Try-On', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

E:\new_downloads_default_folder\anaconda_destination_folder\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
